In [2]:
%matplotlib inline
import pandas as pd
import os
import os.path
import itertools
import matplotlib.pyplot as plt
import math
import pprint
import pprint as pp
import numpy as np

## 1.  LOAD DATA

In [3]:
DATA_PATH = os.path.abspath(os.path.join(os.pardir, "data/gapminder"))
DATA_PATH2 = os.path.abspath(os.path.join(os.pardir, "data"))
Child_death_FILE = "child-death.csv"
GINI_FILE = "gini-complete.csv"
#Year_in_school = "Years in school men 25-34.csv"
FERTILITY_FILE = "teen-fertility.csv"
COUNTRIES_FILE = "3-countries.csv"
fertility_df = pd.read_csv(os.path.join(DATA_PATH, FERTILITY_FILE), sep=";")
gini_df = pd.read_csv(os.path.join(DATA_PATH, GINI_FILE),sep=",")
child_death_df = pd.read_csv(os.path.join(DATA_PATH, Child_death_FILE), sep=";")
countries_df = pd.read_csv(os.path.join(DATA_PATH2, COUNTRIES_FILE), sep=";", usecols = ['Country (en)', "Continent"])

In [4]:
countries_df = countries_df.rename(columns ={"Country (en)" :"countries"})
countries_df.head(3)

,countries,Continent
0,Afghanistan,Asia
1,Egypt,Africa
2,Åland Islands,Europe


In [5]:
fertility_df.head(3)

,countries,Teen-fertility
0,Abkhazia,NaN
1,Afghanistan,115.0
2,Akrotiri and Dhekelia,NaN


In [6]:
gini_df.head(3)

,countries,Gini
0,Afghanistan,27.8
1,Lesotho,63.2
2,South_Africa,62.5


In [7]:
child_death_df.head(3)

,countries,child-death
0,Afghanistan,195.188070
1,Albania,10.526316
2,Algeria,18.090586


In [8]:
df_intermediate = pd.merge(gini_df, child_death_df,  how='inner', on=['countries'])
df_intermediate.head(3)


,countries,Gini,child-death
0,Afghanistan,27.8,195.188070
1,Lesotho,63.2,39.732421
2,Haiti,60.8,45.507790


In [9]:
df_final = pd.merge(df_intermediate, countries_df,  how='left', on=['countries'])
df_final.head(3)

,countries,Gini,child-death,Continent
0,Afghanistan,27.8,195.188070,Asia
1,Lesotho,63.2,39.732421,Africa
2,Haiti,60.8,45.507790,North America


In [10]:
df_final_cols = df_final.columns.tolist()
df_final_cols = df_final_cols[-1:] + df_final_cols[:-1]
df_final = df_final[df_final_cols]
df_final.head(3)


,Continent,countries,Gini,child-death
0,Asia,Afghanistan,27.8,195.188070
1,Africa,Lesotho,63.2,39.732421
2,North America,Haiti,60.8,45.507790


In [11]:
import json
from json import dumps

json_dict = {}
json_dict["name"] = "Json_file"
json_dict['children'] = []
for grp, grp_data in df_final.groupby('Continent'):
    #print ('grp_data',grp_data)
    grp_dict = {}
    #print (grp)
    grp_dict['name'] = grp
    grp_dict['children'] = []
    for cat, cat_data in grp_data.groupby('countries'):
        #print ("cat",cat)
        #print ("cat_data",cat_data)
        
        cat_dict = {}
         
        cat_dict['name'] = cat
        
        cat_dict['children'] = []
        #cat_dict['name'] = cat
        for p, p_data in cat_data.groupby('Gini'):
            cat_dict['gini'] =p//10
            p_data = p_data.drop(['Continent', 'countries'], axis=1).set_index('Gini')
            
            for d in p_data.to_dict(orient='records'):
                for key, value in d.items():
                    d_prime ={}
                    d_prime ["name"] =key
                    d_prime ["size"] =value
                    cat_dict['children'].append(d_prime)
        grp_dict['children'].append(cat_dict)
    json_dict['children'].append(grp_dict)
json_out = dumps(json_dict)
parsed = json.loads(json_out)    

In [12]:
json_out

'{"name": "Json_file", "children": [{"name": "Africa", "children": [{"name": "Algeria", "children": [{"name": "child-death", "size": 18.0905859}], "gini": 3.0}, {"name": "Angola", "children": [{"name": "child-death", "size": 166.79505840000002}], "gini": 4.0}, {"name": "Benin", "children": [{"name": "child-death", "size": 82.27262885}], "gini": 3.0}, {"name": "Botswana", "children": [{"name": "child-death", "size": 14.95224178}], "gini": 6.0}, {"name": "Burundi", "children": [{"name": "child-death", "size": 119.3588838}], "gini": 4.0}, {"name": "Cameroon", "children": [{"name": "child-death", "size": 95.87426717}], "gini": 4.0}, {"name": "Chad", "children": [{"name": "child-death", "size": 153.56249340000002}], "gini": 4.0}, {"name": "Djibouti", "children": [{"name": "child-death", "size": 57.06724010000001}], "gini": 4.0}, {"name": "Egypt", "children": [{"name": "child-death", "size": 8.749506845}], "gini": 3.0}, {"name": "Ethiopia", "children": [{"name": "child-death", "size": 64.608

In [17]:
import io
path = os.path.join(os.pardir, "data/gapminder")
# process Unicode text
with io.open(path+'/'+'gini_child_death_.json','w',encoding='utf8') as f:
    json.dump(parsed, f)

In [14]:
path = os.path.join(os.pardir, "data/gapminder")

'../data/gapminder'